In [1]:
import librosa
import numpy as np
import cv2
from glob import glob
import tqdm
import os

In [2]:
data_root= '/home/philip/data/Keyword_spot/'
join = os.path.join

In [3]:
def audio_to_mel_to_db(audio_file):
    y, sr = librosa.load(audio_file, sr=None)
    mel = librosa.feature.melspectrogram(y,sr,hop_length=256)
    db = librosa.power_to_db(mel, ref=np.min)
    return db

In [4]:
audio_folder = join(data_root,"train/audio/")
valid_list = []
with open(join(data_root,"train/validation_list.txt"),'r') as f:
    for file in f:
        full_filename = os.path.join(audio_folder,file)
        valid_list.append(full_filename)

In [5]:
test_list = []
with open(join(data_root,"train/testing_list.txt"),'r') as f:
    for file in f:
        full_filename = os.path.join(audio_folder,file)
        test_list.append(full_filename)

In [6]:
def del_n(string):
    return string[:-1]
valid_list = list(map(del_n,valid_list))
test_list = list(map(del_n,test_list))

In [7]:
def handle_backgroud_audio(backgroud_audio_folder,target_image_folder,target_width):
    counter = len(glob(target_image_folder))
    for audio_file in glob(backgroud_audio_folder+"***.wav")+glob(backgroud_audio_folder+"*.mp3"):
        audio_len = librosa.get_duration(filename=audio_file)
        for off_set in np.arange(0,audio_len-0.5,0.5):
            y, sr = librosa.load(audio_file, sr=None,offset=off_set,duration=1)
            mel = librosa.feature.melspectrogram(y,sr,hop_length=256)
            db = librosa.power_to_db(mel, ref=np.min)
            counter += 1
            image_path = os.path.join(target_image_folder,str(counter)+".png")
            if db.shape[1]!=target_width:
                db =cv2.resize(db,(target_width,128))
            cv2.imwrite(image_path,db)

In [8]:
handle_backgroud_audio(data_root+"_background_noise_/",data_root+"train/silence",63)

In [9]:
handle_backgroud_audio(data_root+"augment_background/",data_root+"train/silence",63)

In [ ]:
db_length = []
for i in tqdm.tqdm(glob("test/audio/*.wav",recursive=True)):
#     audio_path = i.replace("\\","/")
    audio_path = i
    db = audio_to_mel_to_db(audio_path)
#     if not judge_invalid_data(db):
#         continue
    target_folder = "test_image"
#     if audio_path in valid_list or audio_path in test_list:
#         target_folder = "valid_image"
    target_folder_path = os.path.join("test/",target_folder)
    if not os.path.isdir(target_folder_path):
        os.mkdir(target_folder_path)
    image_path = (audio_path.replace("audio",target_folder)).replace("wav","png")
    new_folder = os.path.dirname(image_path)
    if not os.path.isdir(new_folder):
        os.mkdir(new_folder)
    if db.shape[1]!=63:
        db =cv2.resize(db,(63,128))
    cv2.imwrite(image_path,db)

In [ ]:
for i in tqdm.tqdm(glob(data_root+"train/audio/***/*.wav",recursive=True)):
#     audio_path = i.replace("\\","/")
    audio_path = i
    db = audio_to_mel_to_db(audio_path)
#     if not judge_invalid_data(db):
#         continue
    target_folder = "train_on_all/"
#     if audio_path in valid_list or audio_path in test_list:
#         target_folder = "valid_image"
    target_folder_path = os.path.join(data_root,target_folder)
    if not os.path.isdir(target_folder_path):
        os.mkdir(target_folder_path)
    image_path = (audio_path.replace("audio",target_folder)).replace("wav","png")
    new_folder = os.path.dirname(image_path)
    if not os.path.isdir(new_folder):
        os.mkdir(new_folder)
    if db.shape[1]!=63:
        db =cv2.resize(db,(63,128))
    cv2.imwrite(image_path,db)

In [ ]:
CNN_filter = 16
CNN_layer = 3
CNN_pooling = False
CNN_output_shape = ()

RNN_cell = 96
RNN_layer = 2

In [ ]:
import keras
from model_builder import CRNNnetwork
from keras.layers import InputLayer, Input
from keras.preprocessing import image

In [ ]:
inputs= Input((128,63,1))

In [ ]:
batch_size = 256

In [ ]:
model = CRNNnetwork(inputs)
model.compile(optimizer='adam',loss=keras.losses.categorical_crossentropy,metrics=[keras.metrics.categorical_accuracy])

In [ ]:
model.summary()

In [ ]:
training_data_gen = image.ImageDataGenerator(width_shift_range=0.1)
training_gen = training_data_gen.flow_from_directory(data_root+"train/train_image",class_mode="categorical",target_size=(128,63),color_mode="grayscale")
valid_dataset_dir = os.path.join(data_root,"train/valid_image")
valid_data_gen = keras.preprocessing.image.ImageDataGenerator().flow_from_directory(valid_dataset_dir,
                                                                                    class_mode="categorical",
                                                                                    target_size=(128,63),
                                                                                    color_mode="grayscale",
                                                                                    batch_size=batch_size)

In [ ]:
model.fit_generator(training_gen,steps_per_epoch=100,epochs=2)

In [ ]:
test_audio = "train/_background_noise_/doing_the_dishes.wav"
y, sr = librosa.load(test_audio, sr=None,offset=1,duration=1)
mel = librosa.feature.melspectrogram(y,sr,hop_length=256)
db = librosa.power_to_db(mel,ref=np.min)
librosa.get_duration(filename=test_audio)

In [ ]:
np.arange(0,9.1,0.5)

In [ ]:
def judge_invalid_data(db):
    pixel_count = np.multiply(*db.shape)
    activated = np.sum(db>(255/100))
    return (activated/pixel_count)
#     return activated

In [ ]:
test_file = "/home/philip/data/Keyword_spot/train/audio/bird/4b25f620_nohash_0.wav"
db = audio_to_mel_to_db(test_file)
a = judge_invalid_data(db)

In [ ]:
import speech_recognition as sr
# https://github.com/Uberi/speech_recognition

from os import path
AUDIO_FILE = "train/audio/bird/1bc45db9_nohash_0.wav"
# AUDIO_FILE = path.join(path.dirname(path.realpath(__file__)), "french.aiff")
# AUDIO_FILE = path.join(path.dirname(path.realpath(__file__)), "chinese.flac")

# use the audio file as the audio source
r = sr.Recognizer()
with sr.AudioFile(AUDIO_FILE) as source:
    audio = r.record(source)  # read the entire audio file

# recognize speech using Sphinx
try:
    print("Sphinx thinks you said " + r.recognize_sphinx(audio))
except sr.UnknownValueError:
    print("Sphinx could not understand audio")
except sr.RequestError as e:
    print("Sphinx error; {0}".format(e))

In [ ]:
def detect_by_sphinx(AUDIO_FILE):
    r = sr.Recognizer()
    label = AUDIO_FILE.split("/")[-2]
    with sr.AudioFile(AUDIO_FILE) as source:
        audio = r.record(source)
    try:
        
        if r.recognize_sphinx(audio)== label:
            return ("good",AUDIO_FILE)
        else:
            return ("bad",AUDIO_FILE)
    except sr.UnknownValueError:
        print("Speech Recognition could not understand audio")
        return ("bad",AUDIO_FILE)
    except sr.RequestError as e:
        print("Sphinx error; {0}".format(e))

In [ ]:
def detect_by_google(AUDIO_FILE):
    r = sr.Recognizer()
    with sr.AudioFile(AUDIO_FILE) as source:
        audio = r.record(source)
    try:
        r.recognize_google(audio)
#         ok_file.write(AUDIO_FILE+"\n")
        return ("good",AUDIO_FILE)
    except sr.UnknownValueError:
#         print("Google Speech Recognition could not understand audio")
#         bad_file.write(AUDIO_FILE+"\n")
        return ('bad',AUDIO_FILE)
    except sr.RequestError as e:
        print("Could not request results from Google Speech Recognition service; {0}".format(e))

In [ ]:
good = []
bad = []

In [ ]:
from multiprocessing import Pool
from glob import glob
# ok_file = open("ok.txt","w")
# bad_file = open("bad.txt","w")

p = Pool(4)
files = glob("./train/audio/***/*.wav",recursive=True)
re = p.map(detect_by_sphinx,files)
# bad_file.close()
# ok_file.close()

In [ ]:
good = [_[1] for _ in list(filter(lambda x:x[0]=='good',re))]
bad = [_[1] for _ in list(filter(lambda x:x[0]=='bad',re))]

In [ ]:
len(good)

In [ ]:
len(bad)

In [ ]:
import tqdm

ok_file = open("ok.txt","w")
bad_file = open("bad.txt","w")
for i in bad:
    bad_file.write(i+"\n")
for i in good:
    ok_file.write(i+"\n")
bad_file.close()
ok_file.close()

In [ ]:
GOOGLE_CLOUD_SPEECH_CREDENTIALS = r"""INSERT THE CONTENTS OF THE GOOGLE CLOUD SPEECH JSON CREDENTIALS FILE HERE"""
try:
    print("Google Cloud Speech thinks you said " + r.recognize_google(audio))#, credentials_json=GOOGLE_CLOUD_SPEECH_CREDENTIALS))
except sr.UnknownValueError:
    print("Google Cloud Speech could not understand audio")
except sr.RequestError as e:
    print("Could not request results from Google Cloud Speech service; {0}".format(e))

In [ ]:
import shutil
import os
with open("bad.txt") as bad:
    for i in list(map(del_n,bad)):
        dir_folder = os.path.dirname(i).replace("audio","hard_example")
        if not os.path.isdir(dir_folder):
            os.mkdir(dir_folder)
        shutil.copy(i,dir_folder)